# delta_predict_wind_120E

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
from functools import partial
import glob
import dask
import os
import nc_time_axis
import matplotlib.pyplot as plt
from math import nan
import sys
#from geocat.viz import util as gvutil
import cartopy
import xesmf as xe
import xskillscore as xs
#cartopy.config['pre_existing_data_dir']='/ihesp/shared/cartopy_features'
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import matplotlib.ticker as mticker
%matplotlib inline

from mypyutils import calendar_utils as cal
from mypyutils import stat_utils as stat
from mypyutils import mapplot_utils as maps
from mypyutils import colorbar_utils as cbars
from mypyutils import io_utils as io
from mypyutils import regrid_utils as regrid
from mypyutils import bootstrap_utils as boot

from esp_lab import data_access

In [2]:
import dask
from dask.distributed import wait
dask.__version__

'2024.7.0'

In [65]:
client.shutdown()

In [3]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='15GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=10GB',
        account='P06010014',
        walltime='05:00:00',
        local_directory='/glade/derecho/scratch/liyuanpu/',
        log_directory='/glade/derecho/scratch/liyuanpu/',
        interface='ext',
        job_script_prologue=[
        f'export ESMFMKFILE={os.environ["ESMFMKFILE"]}'])

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
    })
    client = Client(cluster)
    return cluster, client

In [4]:
cluster, client = get_ClusterClient()
cluster.scale(20)

In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/liyuanpu/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/liyuanpu/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.91:45647,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/liyuanpu/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# ERA5

In [8]:
%%time
var = 'U'
datadir = '/glade/campaign/collections/rda/data/d633001/e5.moda.an.pl/????'
casename = 'e5.moda.an.pl.128_131_u.ll025uv'
filetemplate = datadir+'/'+casename+'.*.nc' 

files = sorted(glob.glob(filetemplate))

uera5 = xr.open_mfdataset(files,chunks={})

CPU times: user 463 ms, sys: 121 ms, total: 584 ms
Wall time: 2.83 s


In [10]:
level=uera5["level"]
LAT=uera5['latitude']
LON=uera5['longitude']

In [24]:
tej_era5 = uera5.sel(longitude=120).sel(time=uera5['time'].dt.month.isin([6, 7])).groupby('time.year').mean('time').rename({'year': 'time'}).rename({'latitude': 'lat'}).rename({'longitude': 'lon'}).sel(time=np.arange(1980,2021,1))

In [25]:
tej_era5

<xarray.Dataset> Size: 4MB
Dimensions:   (time: 41, level: 37, lat: 721)
Coordinates:
  * lat       (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level     (level) float64 296B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
    lon       float64 8B 120.0
  * time      (time) int64 328B 1980 1981 1982 1983 1984 ... 2017 2018 2019 2020
Data variables:
    U         (time, level, lat) float32 4MB dask.array<chunksize=(1, 37, 721), meta=np.ndarray>
    utc_date  (time) float64 328B dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.6.1
    CONVERSION_PLATFORM:  Linux casper04 3.10.0-693.21.1.el7.x86_64 #1 SMP We...
    CONVERSION_DATE:      Fri Nov  8 18:03:42 MST 2019
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Fri Nov  8 18:03:52 2019: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 4.7.9 (Homepage = http://n...

In [33]:
lat_new = np.arange(-90,92,2)
level_new = level.sel(level=slice(5, 1000))

In [34]:
tej_era5 = tej_era5.interp(lat=lat_new, level=level_new)

In [35]:
tej_era5.load()

<xarray.Dataset> Size: 1MB
Dimensions:   (time: 41, level: 34, lat: 91)
Coordinates:
    lon       float64 8B 120.0
  * time      (time) int64 328B 1980 1981 1982 1983 1984 ... 2017 2018 2019 2020
  * lat       (lat) int64 728B -90 -88 -86 -84 -82 -80 -78 ... 80 82 84 86 88 90
  * level     (level) float64 272B 5.0 7.0 10.0 20.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    U         (time, level, lat) float64 1MB 0.0001831 7.784 ... 4.911e-05
    utc_date  (time) float64 328B 1.98e+09 1.981e+09 ... 2.019e+09 2.02e+09
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.6.1
    CONVERSION_PLATFORM:  Linux casper04 3.10.0-693.21.1.el7.x86_64 #1 SMP We...
    CONVERSION_DATE:      Fri Nov  8 18:03:42 MST 2019
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Fri Nov  8 18:03:52 2019: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 4.7.9 (Homepage = http://n...

In [65]:
tej_era5.to_netcdf('Isla_delta_predict_wind_section_120E_tej_era5.nc')

# Read in SMYLE-CW3E-L32

In [36]:
initmon=['11','02','09']
lowpath="/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/monthly/"
highpath="/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/monthly/"
savdir="/glade/u/home/liyuanpu/Isla/"

In [37]:
init = initmon[1]
init

'02'

In [38]:
# This preprocessor will return monthly mean field, with first entry removed if DP extension timeseries starts in October
def preproc_cw3e(ds0,nlead,field):
    d0 = cal.time_set_midmonth(ds0,'time')
    mon1 = d0.time.dt.month.values[0]
    if (mon1==10):
        d0 = d0.isel(time=slice(1, nlead+1))
    else:
        d0 = d0.isel(time=slice(0, nlead))
    d0 = d0.assign_coords(L=("time", np.arange(d0.sizes["time"])+1))
    d0 = d0.swap_dims({"time": "L"})
    d0 = d0.reset_coords(["time"])
    d0["time"] = d0.time.expand_dims("Y")
    d0 = d0[[field,'time','lev','lat','lon']].chunk({'L':-1})
    return d0

In [39]:
%%time
# process all 10 ensemble members
var = 'U'
datadir = lowpath
casename = 'b.e21.BSMYLE-CW3E.f09_g17.????-MM.EEE'
filetype = '.cam.h0.'
filetemplate = datadir+var+'/'+casename+filetype+var+'.*.nc' 
ens = np.arange(1,21,1)
nlead = 6
startyears = np.arange(1980,2021,1)
startmonth = 2
chunk = {}
file_list, yrs=io.nested_file_list_by_year(filetemplate, filetype, ens, startyears, startmonth)

CPU times: user 2.97 s, sys: 2.04 s, total: 5.02 s
Wall time: 5.21 s


In [48]:
dple = io.get_monthly_data(filetemplate,filetype,ens,nlead,var,startyears,startmonth,preproc_cw3e,chunks=chunk)

In [49]:
dple = dple.sel(lon = 120).sel(L=slice(4,6)).mean('L').squeeze().rename({"lev": "level"})

In [50]:
dple

<xarray.Dataset> Size: 20MB
Dimensions:  (Y: 41, M: 20, level: 32, lat: 192)
Coordinates:
  * level    (level) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lon      float64 8B 120.0
  * Y        (Y) int64 328B 1980 1981 1982 1983 1984 ... 2017 2018 2019 2020
  * M        (M) int64 160B 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Data variables:
    U        (Y, M, level, lat) float32 20MB dask.array<chunksize=(1, 1, 16, 96), meta=np.ndarray>

In [53]:
tej_low = dple.interp(lat=lat_new, level=level_new).rename({"Y": "time"})

In [63]:
tej_low.load()

<xarray.Dataset> Size: 20MB
Dimensions:  (time: 41, M: 20, level: 34, lat: 91)
Coordinates:
    lon      float64 8B 120.0
  * time     (time) int64 328B 1980 1981 1982 1983 1984 ... 2017 2018 2019 2020
  * M        (M) int64 160B 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
  * lat      (lat) int64 728B -90 -88 -86 -84 -82 -80 -78 ... 80 82 84 86 88 90
  * level    (level) float64 272B 5.0 7.0 10.0 20.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    U        (time, M, level, lat) float64 20MB -3.532 3.763 11.22 ... nan nan

In [64]:
tej_low.to_netcdf('Isla_delta_predict_wind_section_120E_tej_low.nc')

# Read in SMYLE-CW3E-L83

In [55]:
initmon=['11','02','09']
lowpath="/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/monthly/"
highpath="/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/monthly/"
savdir="/glade/u/home/liyuanpu/Isla/"

In [56]:
init = initmon[1]

In [57]:
# This preprocessor will return monthly mean field, with first entry removed if DP extension timeseries starts in October
def preproc_cw3e(ds0,nlead,field):
    d0 = cal.time_set_midmonth(ds0,'time')
    mon1 = d0.time.dt.month.values[0]
    if (mon1==10):
        d0 = d0.isel(time=slice(1, nlead+1))
    else:
        d0 = d0.isel(time=slice(0, nlead))
    d0 = d0.assign_coords(L=("time", np.arange(d0.sizes["time"])+1))
    d0 = d0.swap_dims({"time": "L"})
    d0 = d0.reset_coords(["time"])
    d0["time"] = d0.time.expand_dims("Y")
    d0 = d0[[field,'time','lev','lat','lon']].chunk({'L':-1})
    return d0

In [58]:
%%time
# process all 10 ensemble members, all start dates from 1954-2015:
var = 'U'
datadir = highpath
casename = 'b.e21.BSMYLE-CW3E-L83.f09_g17.????-MM.EEE'
filetype = '.cam.h0.'
filetemplate = datadir+var+'/'+casename+filetype+var+'.*.nc' 
ens = np.arange(1,21,1)
nlead = 6
startyears = np.arange(1980,2021,1)
startmonth = 2
chunk = {}
file_list, yrs=io.nested_file_list_by_year(filetemplate, filetype, ens, startyears, startmonth)
dple = io.get_monthly_data(filetemplate,filetype,ens,nlead,var,startyears,startmonth,preproc_cw3e,chunks=chunk)

CPU times: user 8.83 s, sys: 3.02 s, total: 11.9 s
Wall time: 15.7 s


In [59]:
dple = dple.sel(lon = 120).sel(L=slice(4,6)).mean('L').squeeze().rename({"lev": "level"})

In [60]:
dple

<xarray.Dataset> Size: 52MB
Dimensions:  (Y: 41, M: 20, level: 83, lat: 192)
Coordinates:
  * level    (level) float64 664B 0.008129 0.01833 0.0348 ... 957.5 976.3 992.6
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lon      float64 8B 120.0
  * Y        (Y) int64 328B 1980 1981 1982 1983 1984 ... 2017 2018 2019 2020
  * M        (M) int64 160B 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Data variables:
    U        (Y, M, level, lat) float32 52MB dask.array<chunksize=(1, 1, 42, 96), meta=np.ndarray>

In [61]:
tej_high = dple.interp(lat=lat_new, level=level_new).rename({"Y": "time"})

In [66]:
tej_high.load()

<xarray.Dataset> Size: 20MB
Dimensions:  (time: 41, M: 20, level: 34, lat: 91)
Coordinates:
    lon      float64 8B 120.0
  * time     (time) int64 328B 1980 1981 1982 1983 1984 ... 2017 2018 2019 2020
  * M        (M) int64 160B 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
  * lat      (lat) int64 728B -90 -88 -86 -84 -82 -80 -78 ... 80 82 84 86 88 90
  * level    (level) float64 272B 5.0 7.0 10.0 20.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    U        (time, M, level, lat) float64 20MB -1.037 4.455 9.911 ... nan nan

In [67]:
tej_high.to_netcdf('Isla_delta_predict_wind_section_120E_tej_high.nc')

# statistics

## EQBO

In [68]:
high = xr.open_dataset('Isla_delta_predict_wind_section_120E_tej_high.nc')
low = xr.open_dataset('Isla_delta_predict_wind_section_120E_tej_low.nc')
era5 = xr.open_dataset('Isla_delta_predict_wind_section_120E_tej_era5.nc')

In [69]:
high = tej_high
low = tej_low
era5 = tej_era5

In [70]:
high_clim=high.mean('M').mean('time')
low_clim=low.mean('M').mean('time')
era5_clim= era5.mean('time')

In [71]:
high=high-high_clim
low=low-low_clim
era5=era5- era5_clim

In [73]:
# First bootstrap with replacement across the member dimension (100 samples)
mod_high = high['U'].transpose("M",...).rename({'time':'init_year'}) ; mod_low = low['U'].transpose("M",...).rename({'time':'init_year'}) ; obs = era5['U'].rename({'time':'init_year'})
boot_high_mem = boot.bootgen(mod_high, nboots=100)
boot_low_mem = boot.bootgen(mod_low, nboots=100)

you are using an xarray dataarray
you are using an xarray dataarray


In [75]:
boot_high_mem = boot_high_mem.transpose("init_year",...) # move init_year to the left most dimension for the next round of bootstrapping
boot_high_mem = boot_high_mem.rename({'iboot':'ibootmem'}) # remaining the bootstrap sample dimension
boot_high_mem = boot_high_mem.rename({'isample':'M'}) # renaming the member dimension
    
boot_low_mem = boot_low_mem.transpose("init_year",...) # move init_year to the left most dimension for the next round of bootstrapping
boot_low_mem = boot_low_mem.rename({'iboot':'ibootmem'}) # remaining the bootstrap sample dimension
boot_low_mem = boot_low_mem.rename({'isample':'M'}) # renaming the member dimension

In [76]:
# Mean squared skill score calculation
def calcmsss(mod,obs,dim='init_year'):
    mse_mod = (1./mod[dim].size)*((mod - obs)**2).sum('init_year')
    mse_obs = (1./mod[dim].size)*(obs**2).sum('init_year')
    msss = 1 - (mse_mod / mse_obs)
    # dealing with the levels where low top doesn't have any data
    msss = msss.where( msss != 1, nan)
    return msss  

In [104]:
import sys
# loop over the bootstrap samples from the first round to avoid memory issues
allboots_high_cor=[]
allboots_high_msss=[]
allboots_low_cor=[]
allboots_low_msss=[]
for iboot in np.arange(0,boot_high_mem.ibootmem.size,1): 

        print(iboot) 
        # ---bootstrap high top
        
        
        boot_high_time = boot.bootgen(boot_high_mem.isel(ibootmem=iboot),nboots=25,seed=iboot+1).rename({'isample':'init_year'})
        
        
        
        boot_high_time_stack = boot_high_time
        # compute ensemble mean
        boot_high_time_stack = boot_high_time_stack.mean('M')
        
        
        
        boot_low_time = boot.bootgen(boot_low_mem.isel(ibootmem=iboot),nboots=25,seed=iboot+1).rename({'isample':'init_year'})
        
        boot_low_time_stack = boot_low_time
        # compute ensemble mean
        boot_low_time_stack = boot_low_time_stack.mean('M')
        
        
        # bootstrap the obs with the same seed
        boot_obs = boot.bootgen(obs,  nboots=25, seed=iboot+1).rename({'isample':'init_year'})
       
        boot_obs_stack = boot_obs
        
        cor_high = xr.corr(boot_obs_stack, boot_high_time_stack, dim='init_year')
        allboots_high_cor.append(cor_high)
        
        msss_high = calcmsss(boot_high_time_stack, boot_obs_stack)
        allboots_high_msss.append(msss_high)
        
        cor_low = xr.corr(boot_obs_stack, boot_low_time_stack, dim='init_year')
        allboots_low_cor.append(cor_low)
        
        msss_low = calcmsss(boot_low_time_stack, boot_obs_stack)
        allboots_low_msss.append(msss_low)
        

0
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
1
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
2
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
3
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
4
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
5
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
6
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
7
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
8
you are using an xarray dataarray
you are using an xarray dataarray
you are using an xarray dataarray
9
you are using an xarray dataarray
you are using an xarray data

In [108]:
allboots_high_cor = xr.concat(allboots_high_cor, dim='iboot2')
allboots_high_msss = xr.concat(allboots_high_msss, dim='iboot2')
allboots_high_cor = allboots_high_cor.stack(boot=['iboot','iboot2'])
allboots_high_msss = allboots_high_msss.stack(boot=['iboot','iboot2'])
allboots_low_cor = xr.concat(allboots_low_cor, dim='iboot2')
allboots_low_msss = xr.concat(allboots_low_msss, dim='iboot2')
allboots_low_cor = allboots_low_cor.stack(boot=['iboot','iboot2'])
allboots_low_msss = allboots_low_msss.stack(boot=['iboot','iboot2'])

allboots_dif_cor = allboots_high_cor - allboots_low_cor
allboots_dif_msss = allboots_high_msss - allboots_low_msss

In [110]:
allboots_dif_cor

<xarray.DataArray (level: 34, lat: 91, boot: 2500)> Size: 62MB
array([[[-7.56434870e-02, -2.81682793e-01, -2.65249720e-01, ...,
          2.83746917e-01, -3.41134637e-01,  7.05431241e-02],
        [ 4.86519708e-03,  8.37565178e-02,  6.38601687e-01, ...,
          6.69758532e-01,  7.91773806e-01,  3.93884775e-01],
        [ 6.27120914e-02,  6.36568563e-02,  5.84659035e-01, ...,
          7.09194626e-01,  7.72910187e-01,  4.16898949e-01],
        ...,
        [-1.10581750e-01,  2.74870496e-01,  1.05237856e-01, ...,
          1.74407894e-01,  2.11866940e-01,  1.07692719e-01],
        [-2.84315851e-01,  1.26609246e-01, -2.15587386e-01, ...,
          1.39178093e-02,  7.39173882e-02, -2.67533165e-02],
        [-1.81739922e-02,  1.77786909e-01, -6.31552985e-02, ...,
         -1.73936115e-02,  5.68237077e-02,  3.29470519e-01]],

       [[ 1.02341479e-01, -8.62819168e-02,  1.81983463e-01, ...,
          2.73011698e-01,  1.30117366e-01, -2.50747651e-01],
        [ 3.53164188e-02,  1.30235208e-01,  5.84720106e-01, ...,
          6.70030758e-01,  8.07479297e-01,  3.70457834e-01],
        [ 8.48404027e-02,  1.32306946e-01,  5.30675443e-01, ...,
          7.04324814e-01,  7.99580770e-01,  3.59582082e-01],
...
        [ 2.07487740e-01,  1.58181748e-01,  3.20608731e-01, ...,
          5.07173957e-01,  1.46140782e-01,  9.45779018e-01],
        [ 1.57671741e-01,  1.79919913e-01,  1.53137708e-01, ...,
          3.98306031e-01,  2.83394771e-01,  7.28722722e-01],
        [ 4.74233027e-02, -2.66953923e-01, -3.55664240e-01, ...,
          2.62773807e-01, -4.60963058e-02,  4.17901526e-01]],

       [[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        ...,
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]]])
Coordinates:
  * level    (level) float64 272B 5.0 7.0 10.0 20.0 ... 925.0 950.0 975.0 1e+03
  * lat      (lat) int64 728B -90 -88 -86 -84 -82 -80 -78 ... 80 82 84 86 88 90
  * boot     (boot) object 20kB MultiIndex
  * iboot    (boot) int64 20kB 0 0 0 0 0 0 0 0 0 ... 24 24 24 24 24 24 24 24 24
  * iboot2   (boot) int64 20kB 0 1 2 3 4 5 6 7 8 ... 91 92 93 94 95 96 97 98 99

In [114]:
allboots_high_cor.name = "high_cor"
allboots_high_msss.name = "high_msss"
allboots_low_cor.name = "low_cor"
allboots_low_msss.name = "low_msss"
allboots_dif_cor.name = "dif_cor"
allboots_dif_msss.name = "dif_msss"
datout0 = xr.merge([allboots_high_cor.drop_vars("boot"), allboots_high_msss.drop_vars("boot"),
                       allboots_low_cor.drop_vars("boot"), allboots_low_msss.drop_vars("boot"),
                       allboots_dif_cor.drop_vars("boot"), allboots_dif_msss.drop_vars("boot")], compat='override')
datout0.to_netcdf('Isla_delta_predict_wind_section_120E_result.nc')

In [111]:
min95_high_cor = allboots_high_cor.quantile(0.05, dim='boot') ; min95_high_cor = min95_high_cor.rename('min95_high_cor')
max95_high_cor = allboots_high_cor.quantile(0.95, dim='boot') ; max95_high_cor = max95_high_cor.rename('max95_high_cor')
    
min95_low_cor = allboots_low_cor.quantile(0.05, dim='boot') ; min95_low_cor = min95_low_cor.rename('min95_low_cor')
max95_low_cor = allboots_low_cor.quantile(0.95, dim='boot') ; max95_low_cor = max95_low_cor.rename('max95_low_cor')
    
min95_high_msss = allboots_high_msss.quantile(0.05, dim='boot') ; min95_high_msss = min95_high_msss.rename('min95_high_msss')
max95_high_msss = allboots_high_msss.quantile(0.95, dim='boot') ; max95_high_msss = max95_high_msss.rename('max95_high_msss')
    
min95_low_msss = allboots_low_msss.quantile(0.05, dim='boot') ; min95_low_msss = min95_low_msss.rename('min95_low_msss')
max95_low_msss = allboots_low_msss.quantile(0.95, dim='boot') ; max95_low_msss = max95_low_msss.rename('max95_low_msss')
    
min95_dif_cor = allboots_dif_cor.quantile(0.05, dim='boot') ; min95_dif_cor = min95_dif_cor.rename('min95_dif_cor')
max95_dif_cor = allboots_dif_cor.quantile(0.95, dim='boot') ; max95_dif_cor = max95_dif_cor.rename('max95_dif_cor')
    
min95_dif_msss = allboots_dif_msss.quantile(0.05, dim='boot') ; min95_dif_msss = min95_dif_msss.rename('min95_dif_msss')
max95_dif_msss = allboots_dif_msss.quantile(0.95, dim='boot') ; max95_dif_msss = max95_dif_msss.rename('max95_dif_msss')
datout = xr.merge([min95_high_cor, max95_high_cor,
                       min95_low_cor, max95_low_cor,
                       min95_high_msss, max95_high_msss,
                       min95_low_msss, max95_low_msss,
                       min95_dif_cor, max95_dif_cor,
                       min95_dif_msss, max95_dif_msss], compat='override')


datout.to_netcdf('Isla_delta_predict_wind_section_120E_quantile95.nc')

/glade/work/liyuanpu/conda-envs/env_jupyter_liyp/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


##### 